In [2]:
import pennylane as qml
from lib.block_circs import *
from lib.cr2_pot import *
from lib.dvr1d import *
from lib.dvr2d import *
from lib.vqe import *
from pennylane import numpy as np

# def decompose_hamiltonian_1(params16, spin, N):    
#     mol_params = cr2_params
#     mol_params['name'] = spin
#     dvr_options = {
#         'type': '1d',
#         'box_lims': (params16[0], params16[1]),
#         'dx': (params16[1] - params16[0]) / 32,
#         'count': 32
#     } 
    
#     pot, lims = get_pot_cr2(spin)
    
#     dvr_vqe = DVR_VQE(mol_params, pot)  # Initialize the class with given cr2_params
#     h_dvr = dvr_vqe.get_h_dvr(dvr_options=dvr_options)
    
#     # Decompose the Hamiltonian - assuming qml.pauli_decompose exists or is a placeholder for your decomposition logic
#     h_dvr_pd = qml.pauli_decompose(h_dvr)
    

#     return h_dvr, h_dvr_pd

# params16 = [4 * 1.8897259886, 9 * 1.8897259886]
# N = 5
# q = 2**N
# dvr_options = {
#         'type': '1d',
#         'box_lims': (params16[0], params16[1]),
#         'dx': (params16[1] - params16[0]) /q,
#         'count': q
#     } 
 
# spin = 13
# h_dvr, h_dvr_pd = decompose_hamiltonian_1(params16, spin, N)
# eigvals, eigvecs = np.linalg.eigh(h_dvr)
# print(eigvals)

ModuleNotFoundError: No module named 'seaborn'

In [3]:
def gen_ansatz_op(mol_params, spin, params16):
    from qiskit.quantum_info import SparsePauliOp
    
    from qiskit.circuit.library import ECRGate, SXGate
    mol_params = mol_params.copy()  # create a copy to avoid changing the original mol_params
    dvr_options = {
            'type': '1d',
            'box_lims': (params16[0], params16[1]),
            'dx': (params16[1] - params16[0]) / 32,
            'count': 32,
        }
    
    # obtain the potential for a CR2 at certain radii
    pot, lims = get_pot_cr2(spin)
    
    # perform a dvr vqe to obtain the hamiltonian
    dvr_vqe = DVR_VQE(mol_params, pot)
    h_dvr = dvr_vqe.get_h_dvr(dvr_options, J=0) * hartree
     # Perform a Pauli Decomposition to get the Hamiltonian and get a composition.
    h_dvr_p0 = SparsePauliOp.from_operator(h_dvr)
    print(h_dvr_p0)
    num_qubits = int(np.log2(h_dvr.shape[0]))
    return h_dvr, h_dvr_p0
%pip install penny-lane-qiskit
from qiskit.circuit.library import TwoLocal 
from qiskit.converters import dag_to_circuit


def conversion_to_pennylane(h_dvr_p0):
    coeffs = h_dvr_p0.coeffs
    terms = str(h_dvr_p0._pauli_list)
    H = qml.Hamiltonian(coeffs, term)
    return H 

h_dvr1, h_dvr_p0 = gen_ansatz_op(cr2_params, spin, params16)
H = conversion_to_pennylane(h_dvr_p0)




ERROR: Could not find a version that satisfies the requirement penny-lane-qiskit (from versions: none)
ERROR: No matching distribution found for penny-lane-qiskit
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'qiskit'

In [1]:
# wires = h_dvr_pd.wires
# print(wires)
dev_ideal = qml.device("default.mixed", wires=wires)

# Define the quantum circuit
@qml.qnode(dev_ideal)
def circuit_generator(params):
    for param in params[:-1]:
        for i, wire in enumerate(wires):
            qml.RY(param[i], wires = wire)
            
        for i in range(len(wires)-1):
            qml.CNOT(wires = [wires[i], wires[(i+1)]])
            
        qml.Barrier(wires)   
        for i, wire in enumerate(wires):
            qml.RY(params[-1][i], wires = wire)
    return qml.expval(h_dvr_pd)   



params = np.random.random([3,5])
params = np.array(params, requires_grad=True)

qml.draw_mpl(circuit_generator, decimals = 2)(params)

circuit_generator(params)



NameError: name 'qml' is not defined

In [15]:
from qiskit.quantum_info import SparsePauliOp
# def decompose_hamiltonian_1(cr2_params, params16, spin, N):
#     q = 2**N
    
#     dvr_options = {
#         'type': '1d',
#         'box_lims': (params16[0], params16[1]),
#         'dx': (params16[1] - params16[0]) /q,
#         'count': q
#     } 
    
    
#     pot, lims = get_pot_cr2(spin)
    
#     dvr_vqe = DVR_VQE(cr2_params, pot)  # Initialize the class with given cr2_params
#     h_dvr = dvr_vqe.get_h_dvr(dvr_options=dvr_options)
#     h_dvr_pd = SparsePauliOp.from_operator(h_dvr)
    
#     return h_dvr, h_dvr_pd

# params16 = [5.9*, 9]
# N = 5
# q = 2**N
# dvr_options = {
#         'type': '1d',
#         'box_lims': (params16[0], params16[1]),
#         'dx': (params16[1] - params16[0]) /q,
#         'count': q
#     } 

# spin = 13

# h_dvr2, h_dvr_pd2 = decompose_hamiltonian_1(cr2_params, params16, spin, N)


print(h_dvr_pd)


  (-68.33981825172334) [Z0 I1 I2 I3 I4]
+ (-52.064423767002154) [I0 I1 I2 I3 X4]
+ (-42.07426089200725) [I0 Z1 I2 I3 I4]
+ (-33.58815226284804) [Z0 Z1 I2 I3 I4]
+ (-28.920759253626663) [I0 I1 I2 X3 X4]
+ (-23.143686653956003) [I0 I1 I2 Y3 Y4]
+ (-22.428009191831325) [I0 I1 Z2 I3 I4]
+ (-17.848007743242967) [Z0 I1 Z2 I3 I4]
+ (-15.242453850919015) [I0 I1 X2 X3 X4]
+ (-13.678391132623796) [I0 I1 Y2 Y3 X4]
+ (-11.826896200307315) [I0 I1 Y2 X3 Y4]
+ (-11.431306871890236) [I0 I1 I2 Z3 I4]
+ (-10.732988082305255) [I0 Z1 Z2 I3 I4]
+ (-9.094639775764174) [Z0 I1 I2 Z3 I4]
+ (-8.395319779556612) [Z0 Z1 Z2 I3 I4]
+ (-7.818752071386352) [I0 X1 X2 X3 X4]
+ (-7.424008484060531) [I0 Y1 Y2 X3 X4]
+ (-6.905904956590098) [I0 Y1 X2 Y3 X4]
+ (-5.9495995394485925) [I0 Y1 X2 X3 Y4]
+ (-5.739949076418524) [I0 I1 I2 I3 Z4]
+ (-5.492928447606342) [I0 Z1 I2 Z3 I4]
+ (-4.565859316783477) [Z0 I1 I2 I3 Z4]
+ (-4.299951413465223) [Z0 Z1 I2 Z3 I4]
+ (-3.9567830900514296) [X0 X1 X2 X3 X4]
+ (-3.925642064713119) [I0 I

In [19]:
np.linalg.eigvals(h_dvr_pd)


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

array([-0.00249998])